<div>
<img src="https://i.ibb.co/v3CvVz9/udd-short.png" width="150"/>
    <br>
    <strong>Universidad del Desarrollo</strong><br>
    <em>Magíster en Data Science</em><br>
    <em>Profesora: Maria Paz Raveau</em><br>
    <em>Asignatura: Procesamiento de Lenguaje Natural</em><br>

</div>

## **Proyecto Final PLN: Sesgo en Word Embeddings (WEAT)**
**Curso:** Procesamiento de Lenguaje Natural – Magíster en Data Science UDD (2025) 

*Fecha de Entrega: Martes 02, Septiembre 2025.*

**Estudiantes**: Victor Saldivia Vera, Cristian Tobar, Joaquin Leiva.

### **1. Instrucciones y Enunciado**

Se ha dicho que los modelos de WordEmbeddings contienen sesgos. Se solicita investigar qué
sesgos han sido identificados en la literatura, y buscar al menos una métrica para evaluar alguno de
estos sesgos. Testear esta métrica en algunos de los modelos pre-entrenados en idioma
español y concluir respecto al sesgo encontrado (o no encontrado). 

El entregable es un reporte de las siguientes características:

- **Introducción** sobre sesgos que han sido identificados en modelos de embeddings *(máximo
1200 palabras, 10 puntos)*. Se evaluará el uso de literatura pertinente. Ser informativo:
¿Qué sesgo se han identificados?, ¿Qué corpus fue utilizado?, ¿Con qué métricas? En esta sección deberá
explicitar que sesgo se eligió para testear, qué métrica y con qué modelo pre-entrenado en
español se trabajó. Todo debe estar debidamente referenciado.
- **Método:** explicar qué métrica fue utilizada para evaluar la presencia del sesgo. Se debe explicar la
métrica, no el código. El código se entregará como anexo y no será necesariamente
revisado *(máximo 500 palabras, 5 puntos)*.
- **Resultados/Conclusiones:** Reportar los resultados y concluir respecto a la ideonidad
de la métrica usada, y a la presencia o no de sesgo en el corpus. ¿Son consistentes sus
resultados con los reportados en la literatura? *(700 palabras máximo)*.
- Referencias.

### **2. Breve Introducción**
En este cuaderno Jupyter se implementa el **Word Embedding Association Test (WEAT)** para evaluar la presencia de sesgos de género en *word embeddings* en español.  

Se utiliza como **modelo principal** `fastText` en español (`cc.es.300.vec`) y opcionalmente `SBW` (Spanish Billion Words, word2vec).  

El enfoque está en el **sesgo de género** (hombre ↔ carrera vs mujer ↔ familia).

### **3. Imports y Seeds**

In [23]:
import os, gzip, zipfile, shutil, json, random
from pathlib import Path
from typing import List, Tuple, Dict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from gensim.models import KeyedVectors


In [24]:
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

### **3. Ajustes de Rutas del Proyecto**

In [25]:
# Se sube un nivel para llegar a la raíz del proyecto:
PROJECT_ROOT = Path("..").resolve()

# Carpeta de embeddings y de salidas en la raíz del repositorio
DATA_DIR   = PROJECT_ROOT / "data_embeddings"
OUTPUT_DIR = PROJECT_ROOT / "outputs"

# Se crean las carpetas si no existen
DATA_DIR.mkdir(exist_ok=True, parents=True)
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

print("Raíz del proyecto:", PROJECT_ROOT)
print("Embeddings directorio:", DATA_DIR)
print("Outputs directorio:", OUTPUT_DIR)

Raíz del proyecto: C:\Users\victo\OneDrive\Documentos\Programming-2025\UDD-2025\NLP\Proyecto
Embeddings directorio: C:\Users\victo\OneDrive\Documentos\Programming-2025\UDD-2025\NLP\Proyecto\data_embeddings
Outputs directorio: C:\Users\victo\OneDrive\Documentos\Programming-2025\UDD-2025\NLP\Proyecto\outputs


### **4.  Modelos Pre-entrenados en Español**

Para este proyecto utilizamos embeddings **pre-entrenados**, que son vectores generados a partir de grandes corpus de texto.  

- **fastText (cc.es.300.vec.gz)** → entrenado en *Common Crawl + Wikipedia*, con 300 dimensiones y subpalabras.  
- **SBW word2vec (sbw-300-min5.vec.gz)** → entrenado en *Spanish Billion Words Corpus* (aprox 1.5 Billones de tokens).  

*Observaciones:*

- *Ambos se deben descargar y colocar en la carpeta `data_embeddings/`*
- *El archivo `.gz` se descomprime a `.vec` antes de cargarse con `gensim`*

Se crea un `función` de nombre `gunzip` que descomprime un `.gz` a `.vec` (texto). No hace nada si ya existe el `.vec`.

In [ ]:
# FastText: .gz -> .vec
FASTTEXT_VEC_GZ  = DATA_DIR / "cc.es.300.vec.gz"
FASTTEXT_VEC_TXT = DATA_DIR / "cc.es.300.vec"

# SBW: puede venir como .zip o .txt.zip
# Se busca cualquier zip que empiece con "SBW-vectors"
SBW_ZIP = None
for f in DATA_DIR.glob("SBW-vectors*.zip"):
    SBW_ZIP = f
    break

# Archivo esperado después de descomprimir
SBW_VEC_TXT = DATA_DIR / "SBW-vectors-300-min5.txt"

# (Compatibilidad) SBW legado: .gz -> .vec
SBW_VEC_GZ_LEGACY  = DATA_DIR / "sbw-300-min5.vec.gz"
SBW_VEC_TXT_LEGACY = DATA_DIR / "sbw-300-min5.vec"

def gunzip(src_gz: Path, dst_txt: Path):
    if dst_txt.exists():
        print("Ya existe:", dst_txt)
        return True
    if not src_gz.exists():
        print("No se encuentra:", src_gz)
        return False
    print("Descomprimiendo .gz →", dst_txt.name)
    with gzip.open(src_gz, 'rb') as f_in, open(dst_txt, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
    print("Listo:", dst_txt)
    return True

def unzip(src_zip: Path, expected_txt: Path, dest_dir: Path):
    if expected_txt.exists():
        print("Ya existe:", expected_txt)
        return True
    if not src_zip or not src_zip.exists():
        print("No se encontró el archivo zip:", src_zip)
        return False
    print("Descomprimiendo .zip →", dest_dir)
    with zipfile.ZipFile(src_zip, 'r') as zf:
        zf.extractall(dest_dir)
        names = zf.namelist()
    if expected_txt.exists():
        print("Listo:", expected_txt)
        return True
    else:
        print("Extraído, pero no se encontro el archivo esperado:", expected_txt.name)
        print("Archivos en el zip:", names[:10], "... (máximo 10)")
        return False

# PROCESO
# 1. fastText (.gz → .vec)
_ = gunzip(FASTTEXT_VEC_GZ, FASTTEXT_VEC_TXT)

# 2. SBW Kaggle (.zip/.txt.zip → .txt)
ok_sbw = False
if SBW_ZIP:
    ok_sbw = unzip(SBW_ZIP, SBW_VEC_TXT, DATA_DIR)

# 3. Si no hubo zip Kaggle, intentar legado (.gz → .vec)
if not ok_sbw and SBW_VEC_GZ_LEGACY.exists():
    ok_sbw = gunzip(SBW_VEC_GZ_LEGACY, SBW_VEC_TXT_LEGACY)

print("\nResumen:")
print(" - fastText .vec:", FASTTEXT_VEC_TXT.exists(), FASTTEXT_VEC_TXT)
print(" - SBW (Kaggle .txt):", SBW_VEC_TXT.exists(), SBW_VEC_TXT)
print(" - SBW (legado .vec):", SBW_VEC_TXT_LEGACY.exists(), SBW_VEC_TXT_LEGACY)




Descomprimiendo .gz → cc.es.300.vec
Listo: C:\Users\victo\OneDrive\Documentos\Programming-2025\UDD-2025\NLP\Proyecto\data_embeddings\cc.es.300.vec
Descomprimiendo .zip → C:\Users\victo\OneDrive\Documentos\Programming-2025\UDD-2025\NLP\Proyecto\data_embeddings
Listo: C:\Users\victo\OneDrive\Documentos\Programming-2025\UDD-2025\NLP\Proyecto\data_embeddings\SBW-vectors-300-min5.txt

Resumen:
 - fastText .vec: True C:\Users\victo\OneDrive\Documentos\Programming-2025\UDD-2025\NLP\Proyecto\data_embeddings\cc.es.300.vec
 - SBW (Kaggle .txt): True C:\Users\victo\OneDrive\Documentos\Programming-2025\UDD-2025\NLP\Proyecto\data_embeddings\SBW-vectors-300-min5.txt
 - SBW (legado .vec): False C:\Users\victo\OneDrive\Documentos\Programming-2025\UDD-2025\NLP\Proyecto\data_embeddings\sbw-300-min5.vec
